In [27]:
import pandas as pd
import numpy as np
import yaml
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, plot_estimation

##### 1. Load all data that is needed

In [28]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [29]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

##### 2. Define all possible equations for mu and qs

The growth rate and the substrate uptake rate are dependend of each other - meaning that if we define a equation for **mu**, then **qs** is defined by mu with $qs=mu/Yxs$. Contrarily, when **qs** is defined by an ODE, then **mu** is calculated with $mu = qs * Yxs$.

In [30]:
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks) # -- MONOD
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) # -- LOGISTIC
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC + INHIBITION
mu4 = lambda qs, Yxs: qs * Yxs # for changing qs

In [31]:
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose) # -- MONOD
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met)) # -- MONOD + NON COMPETITIVE INHIBITION
qs2 = lambda qs_max, c_glucose, Ks_qs, c_biomass, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(c_biomass * lag))) # -- MONOD + METABOLIZED GLU
qs3 = lambda mu, Yxs: mu / Yxs # for changing mu

In [32]:
mu_all=[mu0, mu1, mu2, mu3, mu4]
qs_all=[qs0, qs1, qs2, qs3]

##### 3. Define functions that are needed

In [33]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

In [ ]:
def sensitivity_analysis(rmse):

    # Define your fermentation model function here
    ## imported

    # Define the number of parameters and the parameter ranges
    num_params = 9
    param_ranges = np.array([[0.1, 0.5], [1.0, 10.0], [0.01, 1.0], [0.01, 1.0], [0.01, 1.0], [0.1, 0.5], [0.1, 1.0], [0.01, 1.0], [0.0, 1.0]])  # Adjust the ranges as needed

    # Define the number of Sobol' samples
    num_samples = 1000  # Adjust the number of samples as desired

    # Generate the Sobol' samples using Saltelli's sampling
    param_samples = saltelli.sample(param_ranges, num_samples, calc_second_order=False)

    # Evaluate the model for each set of parameter samples
    model_outputs = np.zeros(num_samples)
    for i in range(num_samples):
        model_outputs[i] = fermentation_model(param_samples[i])

    # Perform Sobol' analysis
    sobol_indices = sobol.analyze(param_ranges, model_outputs, calc_second_order=False, print_to_console=False)

    # Get the first-order indices and parameter names
    first_order_indices = sobol_indices['S1']
    parameter_names = [f'Parameter {i+1}' for i in range(num_params)]

    # Sort the parameter names based on the first-order indices
    sorted_indices = np.argsort(first_order_indices)[::-1]
    sorted_parameter_names = [parameter_names[i] for i in sorted_indices]

    # Print the ranked parameter names and their corresponding first-order indices
    print("Parameter Rank:\n")
    for i, param_name in enumerate(sorted_parameter_names):
        print(f"{i+1}. {param_name}: {first_order_indices[sorted_indices[i]]}")

##### 4. Define set of parameters (3 Versions)

Version 1: Define all parameters manually in the config/parameters.yml file

In [34]:
## not needed anymore
# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']
est_Ks = param['est_ks']

Version 2: Apply LH sampling on all parameters

In [35]:
# Set the number of samples and parameters
num_samples = 50
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.1, 10.0],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.2, 0.6],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.5],   # Range for parameter 8 - m_s
    [0.001, 0.02],  # Range for parameter 9 - lag
]

Version 3: Use some defined parameters and some LH samples

In [45]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.35, 0.35],    # Range for parameter 1 mu_max
    [15, 25],       # Range for parameter 2 X_max
    [0.1, 20.0],    # Range for parameter 3 - Ks
    [0.1, 20.0],    # Range for parameter 4 - Ks_qs
    [0.1, 20.0],     # Range for parameter 5 - Ki
    [0.3, 0.5],     # Range for parameter 6 - Yxs
    [0.1, 1.5],     # Range for parameter 7 - qs_max
    [0.0, 0.2],   # Range for parameter 8 - m_s
    [0.001, 1.0],  # Range for parameter 9 - lag
]

In [46]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 9)

##### 5. Run it

In [47]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        # Define the combination of equations
        mu_eq = mu_all[i]; num_mu = i

        # mu 0 to mu3 are combined with qs3 while for mu4 all qs except for qs3 are combined
        if i != 4:
            j = 3
            qs_eq = qs_all[j]; num_qs = j   

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

            if rmse <= float(6):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
        
        elif i == 4:
            for j in range(3):
                qs_eq = qs_all[j]; num_qs = j   

                # Make the predictions and calculate the error
                rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
                # save the parameters in a dataframe
                append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
                rmse_one_set.append(append_list)

                # Make a plot and save it
                title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
                plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

                if rmse <= float(6):
                    plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/new_eq/data.csv')

df_all_sets.head(10)

,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
478,68,2,3,0.35,24.547936,1.666950,3.691989,2.197854,0.378989,0.382596,0.197396,0.272319,5.198
72,10,2,3,0.35,23.413504,4.169900,17.897366,8.609992,0.350724,0.238254,0.187052,0.715350,5.685
274,39,1,3,0.35,23.825404,18.711777,1.708227,7.567798,0.364503,1.361970,0.199160,0.753443,6.150
71,10,1,3,0.35,23.413504,4.169900,17.897366,8.609992,0.350724,0.238254,0.187052,0.715350,6.300
477,68,1,3,0.35,24.547936,1.666950,3.691989,2.197854,0.378989,0.382596,0.197396,0.272319,6.381
587,83,4,2,0.35,18.956682,9.459776,5.103661,18.843122,0.453433,1.120239,0.174707,0.085295,7.378
435,62,1,3,0.35,23.239969,10.472926,19.692065,15.242796,0.406273,0.584272,0.170524,0.694069,7.657
331,47,2,3,0.35,22.768550,2.673059,19.557047,0.951670,0.311788,0.656071,0.083040,0.041515,7.701
694,99,1,3,0.35,19.240561,6.394107,16.666742,10.222681,0.309867,1.095926,0.180294,0.888372,7.707
555,79,2,3,0.35,19.994535,2.264073,5.306680,6.187673,0.395177,0.815333,0.191317,0.336369,7.930


In [48]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')